In [0]:
%pip install pyyaml
%restart_python

In [0]:
email = "me@org.com"

# For Azure
compute = "Standard_NC4as_T4_v3"

notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
directory_path = '/'.join(notebook_path.split('/')[:-1])
default_yaml_path = "/Workspace"+directory_path+"/../resources/setup_workflow.yaml"
notebooks_path = "/Workspace"+directory_path

In [0]:
import re
with open(default_yaml_path, 'r') as file:
    yaml_content = file.read()

updated_yaml_content = re.sub(r'<email>', email, yaml_content)
updated_yaml_content = re.sub(r'<notebooks_path>', notebooks_path, updated_yaml_content)
updated_yaml_content = re.sub(r'<compute>', compute, updated_yaml_content)

print(updated_yaml_content)

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.jobs import JobSettings
from typing import Optional
import yaml

def create_job_from_yaml(yaml_path: Optional[str] = None, yaml_str: Optional[str] = None):
    if yaml_path is not None:
        with open(yaml_path) as f:
            config = yaml.safe_load(f)
    elif yaml_str is not None:
        config = yaml.safe_load(yaml_str)
    else:
        raise ValueError("Either yaml_path or yaml_str must be provided")
    
    outer_name = [k for k in config['resources']['jobs'].keys()][0]
  
    # Full job settings deserialization
    job_settings = JobSettings.from_dict(config['resources']['jobs'][outer_name])
    
    # instantiate the client
    w = WorkspaceClient()
    
    # create a job just with name
    creation_info = w.jobs.create(name='new created job')
    # now use job settings object to populate
    w.jobs.reset(
        job_id=creation_info.job_id,
        new_settings=job_settings,
    )
    return creation_info.job_id

try:
    job_id = create_job_from_yaml(yaml_str = updated_yaml_content)
    print(f"Created job {job_id}")
except Exception as e:
    print(f"Job creation failed: {e}")

In [0]:
w = WorkspaceClient()
run_by_id = w.jobs.run_now_and_wait(job_id=job_id).result()
# clean up the no longer needed job
w.jobs.delete(job_id=job_id)